<a href="https://colab.research.google.com/github/ltdaovn/NLP-with-Python/blob/master/Fake_News_LogReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import re
import pandas as pd
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.linear_model import LogisticRegressionCV

!wget -c "https://raw.githubusercontent.com/ltdaovn/dataset/master/corona_fake.csv"

df = pd.read_csv('corona_fake.csv')

df.loc[df['label'] == 'Fake', ['label']] = 'FAKE'
df.loc[df['label'] == 'fake', ['label']] = 'FAKE'
df.loc[df['source'] == 'facebook', ['source']] = 'Facebook'
df.text.fillna(df.title, inplace=True)

df.loc[5]['label'] = 'FAKE'
df.loc[15]['label'] = 'TRUE'
df.loc[43]['label'] = 'FAKE'
df.loc[131]['label'] = 'TRUE'
df.loc[242]['label'] = 'FAKE'

df = df.sample(frac=1).reset_index(drop=True)
df.title.fillna('missing', inplace=True)
df.source.fillna('missing', inplace=True)

df['title_text'] = df['title'] + ' ' + df['text']

--2021-08-29 13:12:26--  https://raw.githubusercontent.com/ltdaovn/dataset/master/corona_fake.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [4]:
df['label'].value_counts()

TRUE    586
FAKE    578
Name: label, dtype: int64

In [5]:
df.head()

,title,text,source,label,title_text
0,"I have asthma. If I get COVID-19, am I more li...","Yes, asthma may increase your risk of getting ...",https://www.health.harvard.edu/,TRUE,"I have asthma. If I get COVID-19, am I more li..."
1,Is coronavirus a manufactured bioweapon that C...,"In 2019, a mysterious shipment sent from Canad...",https://www.naturalnews.com/,FAKE,Is coronavirus a manufactured bioweapon that C...
2,Coronavirus: A\nCanadian-American study could ...,A Canadian-U.S. study showed that “black peop...,http://www.24jours.com,FAKE,Coronavirus: A\nCanadian-American study could ...
3,Coronavirus Coverup – Exclusive Interview Of F...,Coronavirus Coverup – Exclusive Interview Of F...,https://greatgameindia.com/,FAKE,Coronavirus Coverup – Exclusive Interview Of F...
4,"BILL GATES, ROCKEFELLER AND CO ASPIRE TO A POP...",We may be looking at a complete collapse of ou...,https://journal-neo.org/,FAKE,"BILL GATES, ROCKEFELLER AND CO ASPIRE TO A POP..."


In [ ]:
df['title_text'][50]

In [7]:
def preprocessor(text):
    
    text = re.sub('<[^>]*>', '', text)
    text = re.sub(r'[^\w\s]','', text)
    text = text.lower()

    return text

In [8]:
df['title_text'] = df['title_text'].apply(preprocessor)

In [ ]:
df['title_text'][50]

In [10]:
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [11]:
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None,
                        tokenizer=tokenizer_porter,
                        use_idf=True,
                        norm='l2',
                        smooth_idf=True)
X = tfidf.fit_transform(df['title_text'])
y = df.label.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.5, shuffle=False)

clf = LogisticRegressionCV(cv=5, scoring='accuracy', random_state=0, n_jobs=-1, verbose=3, max_iter=300).fit(X_train, y_train)

fake_news_model = open('fake_news_model.sav', 'wb')
pickle.dump(clf, fake_news_model)
fake_news_model.close()

In [ ]:
filename = 'fake_news_model.sav'
saved_clf = pickle.load(open(filename, 'rb'))

saved_clf.score(X_test, y_test)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
y_pred = clf.predict(X_test)
print("---Test Set Results---")
print("Accuracy with logreg: {}".format(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))